In [12]:
from dbt_utils import get_sql_improved

override_vars = {
    'stats_per_url_day': {'session_gap_seconds': '1234', 'test_variable': 'X'},
    'event_id_url': {'session_gap_seconds': '123', 'test_variable': 'Y'},
    'sessionized_data': {'session_gap_seconds': '12', 'test_variable': 'Z'},
}
sql0 = get_sql_improved(model='stats_per_url_day',
                        vars={'feature_table': 'basic_features'},
                        depth_ephemeral_models=None,
                        model_specific_vars=override_vars)

Compiling model.objectiv.stats_per_url_day
Compiling model.objectiv.sessionized_data
Compiling model.objectiv.event_id_url


In [13]:
print(sql0)


with __dbt__CTE__extracted_contexts as (
SELECT *,
        value->>'event' AS event,
        JSON_EXTRACT_PATH(value, 'global_contexts') AS global_contexts,
        JSON_EXTRACT_PATH(value, 'location_stack') AS location_stack,
        JSON_EXTRACT_PATH(value, 'time') AS time,
        JSON_EXTRACT_PATH(value, 'events') AS events
 FROM "objectiv"."public"."data"
),  __dbt__CTE__hashed_features as (
WITH selected_stacks AS
(
SELECT   event_id,
         Array_to_string(Array_agg(Cast(x AS TEXT)),',') AS stack_selection,
         Array_to_json(Array_agg(Row_to_json(x))) as selected_stack_location
FROM     __dbt__CTE__extracted_contexts,
         json_to_recordset(location_stack) AS x(_context_type text,id text)
GROUP BY event_id
ORDER BY event_id
)
SELECT *,
       md5(concat(stack_selection,event)) as feature_hash
FROM __dbt__CTE__extracted_contexts
JOIN selected_stacks USING (event_id)
),  __dbt__CTE__sessionized_data as (
--
--
-- TEST VAR: Z - Table "objectiv"."public"."sessionized_data